<h1>Paris restaurant analysis</h1>

<h2>Business Problem Description</h2>

<p>France is definitively the country of food and drinks</p>
<p>Having lived in Paris for a big part of my life, I somehow realised that each neighborhood have a singularity regarding food. Each neighborhood has a diverse range of cuisine from all around the world, but it seems that some neighborhoods are more keen to have a particular type of cuisine than others.</p>
<p>Thinking about this, it raises some interesting questions for someone that wants to open a food business:</p>
<ul>
    <li>What are the more and less common specialities of food in the city of Paris ?</li>
    <li>What is each neighborhood food "speciality" and why ? Can we find some clusters and patterns ?</li>
    <li>When opening a restaurant, in which area should we open this new business depending on the type of food ? Can we find the best location for each type of food ?</li>
</ul>
<p>We are going to look for answers to these questions in this analysis</p>
<p>Having answers to these questions can be trully helpfull for new investors that want to open their first restaurant in Paris to make their investment successfull and optimizing profitability of their future businesses.</p>


<h2>Our data</h2>

<p>The city of Paris is basically segmented into 20 districts that each has a postal code going from 75001 to 75020. We are first going to use this districts to define our areas of analysis and see if we can find meaningful results. We are also going to use open data that we can find on the web. Depending on this we are going to adapt our approach to maximise the meaning of our results.</p>

<ul>
    <li>The coordinates of each district will first be obtained using the <b>pgeocode</b> library, which is easy to use and return coordinates of the lacation with a postal code as input</li>
    <li>We also have a dataset of Paris districts and locations that I found in the website <b>Paris data</b></li>
    <li>The information about restaurants will be obtained with the <b>Foursquare API</b> and loaded in a pandas dataframe</li>
    <li>For each neighborhood we will be getting the list of restaurants in a 1000m radius and conduct analysis on this data to define the food particularities of each neighborhood. We will try to find the best locations for our restaurant using K-Neighborhood clustering with <b>Numpy</b> and <b>Scikit Learn</b></li> 
</ul>

<h2>Methodology</h2>

<h3>Defining the neighborhoods</h3>

<p>First let's import the necessary libraries in our notebook:<p>

In [901]:
import pandas as pd
import numpy as np
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from  pandas import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library
from folium import plugins
from folium.plugins import HeatMap
import pgeocode #library to get coordinates with a postal code
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.neighbors import NearestNeighbors

<p>Let's create a dataframe which each row containing data about our 20 districts:</p>

In [902]:
district_list = []
for i in range (1,21):
    district_list.append("District "+str(i))
district_df = pd.DataFrame(district_list)
district_df.columns = ['District']
#district_df

<p>Let's add the postal codes of each district to our dataframe:</p>

In [903]:
postal_code = []
code = 75001
for i in range (1,21):
    postal_code.append(code)
    code +=1

district_df['Postal Code'] = postal_code
district_df.head()

,District,Postal Code
0,District 1,75001
1,District 2,75002
2,District 3,75003
3,District 4,75004
4,District 5,75005


<p>Now let's get the coordinates of each District in Paris using pgeocode:</p>

In [904]:
#For example first district is 75001
nomi = pgeocode.Nominatim('fr')
nomi.query_postal_code("75001")

postal_code                 75001
country_code                   FR
place_name        Paris 01, Paris
state_name          Île-de-France
state_code                   11.0
county_name                 Paris
county_code                    75
community_name              Paris
community_code                751
latitude                  48.8592
longitude                 2.34525
accuracy                        5
Name: 0, dtype: object

In [905]:
latitude = []
longitude = []
#Looping through the postal codes
for i in district_df['Postal Code']:
    data = nomi.query_postal_code(i)
    latitude.append(data.latitude)
    longitude.append(data.longitude)
print(latitude)
print(longitude)

[48.8592, 48.8655, 48.8637, 48.8601, 48.8448, 48.8534, 48.8565, 48.8763, 48.8718, 48.8709, 48.8574, 48.8412, 48.8322, 48.8331, 48.8412, 48.8637, 48.8835, 48.8925, 48.8817, 48.8646]
[2.34525, 2.3457, 2.35515, 2.34975, 2.34795, 2.3394000000000004, 2.3349, 2.33355, 2.3443500000000004, 2.35245, 2.3641500000000004, 2.3682, 2.35245, 2.3376, 2.3245500000000003, 2.31285, 2.33535, 2.3466, 2.3655, 2.3736]


<p>We have our coordinates, let's add them to our dataframe:</p>

In [906]:
district_df['Latitude'] = latitude
district_df['Longitude'] = longitude
district_df.head()

,District,Postal Code,Latitude,Longitude
0,District 1,75001,48.8592,2.34525
1,District 2,75002,48.8655,2.34570
2,District 3,75003,48.8637,2.35515
3,District 4,75004,48.8601,2.34975
4,District 5,75005,48.8448,2.34795


<p>So we have our dataframe with the necessary coordinates for each district. Let's plot the data using Folium to have an overview of the positioning of our districts.</p>

<p>First, let's get the global coordinates of Paris to position our map:</p>

In [907]:
address = 'Paris'

geolocator = Nominatim(user_agent="paris")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


<p>Now, let's plot our graph:</p>

In [910]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, district in zip(district_df['Latitude'], district_df['Longitude'], district_df['District']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)
map_clusters

<p>As we can see on the map, we have two problems here:</p>
    <ul>
        <li>We don't have enough data points to cover the entire Paris area
        <li>Those points are not equally distributed around the map
    </ul>
<p>Therefore, we will need to consider other options to correctly map our Paris neighborhoods.</p>  

<p>Luckilly, the city of Paris has a website "Paris data" (<a href = 'https://opendata.paris.fr'>link</a>) that contains a public dataset of all administrative neighborhoods of Paris with all the information we need ! Let's import the file and create a dataset with this data:</p>

In [911]:
district_detail = pd.read_csv(r'quartier_paris.csv',sep=';', lineterminator='\r')
district_detail.head()

,N_SQ_QU,C_QU,C_QUINSEE,L_QU,C_AR,N_SQ_AR,PERIMETRE,SURFACE,Geometry X Y,Geometry
0,\n750000015,15.0,7510403.0,Arsenal,4.0,750000004.0,2878.559656,4.872649e+05,"48.851585175,2.36476795387","{""type"": ""Polygon"", ""coordinates"": [[[2.368512..."
1,\n750000018,18.0,7510502.0,Jardin-des-Plantes,5.0,750000005.0,4052.729521,7.983894e+05,"48.8419401934,2.35689388962","{""type"": ""Polygon"", ""coordinates"": [[[2.364561..."
2,\n750000039,39.0,7511003.0,Porte-Saint-Martin,10.0,750000010.0,3245.891413,6.090347e+05,"48.8712446509,2.36150364735","{""type"": ""Polygon"", ""coordinates"": [[[2.363917..."
3,\n750000043,43.0,7511103.0,Roquette,11.0,750000011.0,4973.010557,1.172087e+06,"48.8570640408,2.38036406173","{""type"": ""Polygon"", ""coordinates"": [[[2.379720..."
4,\n750000046,46.0,7511202.0,Picpus,12.0,750000012.0,18261.910318,7.205014e+06,"48.8303592424,2.42882681508","{""type"": ""Polygon"", ""coordinates"": [[[2.411249..."


<p>We have a bunch of data, including perimeter, surface, delimitations, coordinates, insee nomenclature. Let's keep in our dataframe the area names, district number, perimeter, surface and coordinates and rename the columns:</p>

In [912]:
district_detail = district_detail.drop(columns=['N_SQ_QU', 'C_QUINSEE', 'N_SQ_AR'])
district_detail = district_detail.rename(columns={"C_QU": "Neighborhood Number", "L_QU": "Neighborhood Name", "C_AR": "District", "PERIMETRE": "Perimeter", "SURFACE": "Surface"})
district_detail.head()

,Neighborhood Number,Neighborhood Name,District,Perimeter,Surface,Geometry X Y,Geometry
0,15.0,Arsenal,4.0,2878.559656,4.872649e+05,"48.851585175,2.36476795387","{""type"": ""Polygon"", ""coordinates"": [[[2.368512..."
1,18.0,Jardin-des-Plantes,5.0,4052.729521,7.983894e+05,"48.8419401934,2.35689388962","{""type"": ""Polygon"", ""coordinates"": [[[2.364561..."
2,39.0,Porte-Saint-Martin,10.0,3245.891413,6.090347e+05,"48.8712446509,2.36150364735","{""type"": ""Polygon"", ""coordinates"": [[[2.363917..."
3,43.0,Roquette,11.0,4973.010557,1.172087e+06,"48.8570640408,2.38036406173","{""type"": ""Polygon"", ""coordinates"": [[[2.379720..."
4,46.0,Picpus,12.0,18261.910318,7.205014e+06,"48.8303592424,2.42882681508","{""type"": ""Polygon"", ""coordinates"": [[[2.411249..."


<p>Let's erase the null values in the dataframe:</p>

In [913]:
district_detail = district_detail.dropna()
district_detail.shape

(80, 7)

<p>Finally let's separate the X Y coordinates into two columns Latitude and Longitude, erase the Geometry column and reorder the columns:</p>

In [914]:
district_detail[['Latitude','Longitude']] = district_detail['Geometry X Y'].str.split(',',expand=True)
district_detail = district_detail.drop(columns=['Geometry X Y'])
district_detail.head()

,Neighborhood Number,Neighborhood Name,District,Perimeter,Surface,Geometry,Latitude,Longitude
0,15.0,Arsenal,4.0,2878.559656,4.872649e+05,"{""type"": ""Polygon"", ""coordinates"": [[[2.368512...",48.851585175,2.36476795387
1,18.0,Jardin-des-Plantes,5.0,4052.729521,7.983894e+05,"{""type"": ""Polygon"", ""coordinates"": [[[2.364561...",48.8419401934,2.35689388962
2,39.0,Porte-Saint-Martin,10.0,3245.891413,6.090347e+05,"{""type"": ""Polygon"", ""coordinates"": [[[2.363917...",48.8712446509,2.36150364735
3,43.0,Roquette,11.0,4973.010557,1.172087e+06,"{""type"": ""Polygon"", ""coordinates"": [[[2.379720...",48.8570640408,2.38036406173
4,46.0,Picpus,12.0,18261.910318,7.205014e+06,"{""type"": ""Polygon"", ""coordinates"": [[[2.411249...",48.8303592424,2.42882681508


<p>Now we have a clean dataset of the Paris Neighborhoods that we will use to conduct our analysis.</p>

<p>Let's visualize our list of neighborhoods:</p>

In [917]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, neighborhood, district in zip(district_detail['Latitude'], district_detail['Longitude'], district_detail['Neighborhood Name'], district_detail['District']):
    label = '{}, {}'.format(neighborhood, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)
map_clusters

<p>Much better ! We now have a complete list of Paris 80 Neighborhoods that are covering most of Paris areas. We can use this list to conduct our analysis.</p>

<h3>Getting restaurant data</h3>

<p>Now let's use Foursquare to get the list of restaurant that are in a radius of 400m of our neighborhoods datapoints.</p>
<p>Let's start with our first neighborhood:</p>

In [918]:
neighborhood_lat = district_detail['Latitude'].iloc[0]
neighborhood_lon = district_detail['Longitude'].iloc[0]
print(neighborhood_lat)
print(neighborhood_lon)

48.851585175
2.36476795387


In [919]:
CLIENT_ID = 'HLBVAQVRSCO0HJKKDGPBFTORX2AXTJBNNFTSQKTW5GXX4WAX' 
CLIENT_SECRET = 'FHUAZGABZ5130R1SOLY4SFEZWBAHIN4M2G2V0QUFUONGE205' 
VERSION = '20180605' 
LIMIT = 1000

In [920]:
radius = 1000
url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_lon, 
    radius, 
    LIMIT)

results = requests.get(url).json()
#results

In [921]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [922]:
venues = results['response']['venues']
    
nearby_restaurants = json_normalize(venues) # flatten JSON

# filter columns
#filtered_columns = nearby_venues['name', 'categories', 'location.lat', 'location.lng']
nearby_restaurants =nearby_restaurants[['name', 'categories', 'location.lat', 'location.lng']]

# filter the category for each row
nearby_restaurants['categories'] = nearby_restaurants.apply(get_category_type, axis=1)

#rename columns
nearby_restaurants = nearby_restaurants.rename(columns={"location.lat": "latitude", "location.lng": "longitude"})

#nearby_venues
nearby_restaurants.head()

,name,categories,latitude,longitude
0,The Grilled Cheese Factory,Sandwich Place,48.852796,2.367617
1,Berthillon,Ice Cream Shop,48.851705,2.356706
2,L'As du Fallafel,Falafel Restaurant,48.857414,2.359078
3,Nuance Café,Coffee Shop,48.845088,2.354891
4,Les Nautes,Gastropub,48.852231,2.360306


In [923]:
nearby_restaurants.shape

(50, 4)

<p>Let's append the list of restaurants for all locations to see which type of retaurants we are getting and what filters we can apply.</p>

In [924]:
restaurant_list_det = pd.DataFrame(columns =['name', 'categories', 'latitude', 'longitude'])

In [925]:
radius = 1000
for i in range(1,len(district_detail.index)):
    neighborhood_lat = district_detail['Latitude'].iloc[i]
    neighborhood_lon = district_detail['Longitude'].iloc[i]
    url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_lon, 
    radius, 
    LIMIT)

    results = requests.get(url).json()
    venues = results['response']['venues']
    nearby_rest = json_normalize(venues) # flatten JSON

    try:
        nearby_rest =nearby_rest[['name', 'categories', 'location.lat', 'location.lng']]
        nearby_rest['categories'] = nearby_rest.apply(get_category_type, axis=1)
        nearby_rest = nearby_rest.rename(columns={"location.lat": "latitude", "location.lng": "longitude"})
        restaurant_list_det = restaurant_list_det.append(nearby_rest)
    except:
        print('No data for point: ',i)

restaurant_list_det.head()

,name,categories,latitude,longitude
0,Salon de Thé de la Grande Mosquée de Paris,Tea Room,48.841719,2.355626
1,Au P'tit Grec,Creperie,48.842858,2.349721
2,Le Fournil de Mouffetard - Maison Morange,Bakery,48.840198,2.349847
3,La Maison d'Isabelle,Bakery,48.850007,2.348443
4,Bioburger,Burger Joint,48.833669,2.353680


In [926]:
restaurant_list_det = restaurant_list_det.drop_duplicates()
restaurant_list_det.shape

(1411, 4)

<h3>Cleaning the data</h3>

<p>So we have a list of 1383 locations, let's define a dataset where we find the occurence for each distinct food categories.</p>

In [927]:
#count occurences of categories
restaurant_list_agg = restaurant_list_det.groupby(['categories'],as_index=False).count()
#drop latitude and longitude
restaurant_list_agg = restaurant_list_agg.drop(columns=['latitude', 'longitude'])
#rename column and setting index
restaurant_list_agg = restaurant_list_agg.rename(columns={"name": "count"})
#restaurant_list_agg = restaurant_list_agg.set_index('categories') 
#sorting by count
restaurant_list_agg = restaurant_list_agg.sort_values(by=['count'], ascending=False)
pd.set_option('display.max_rows', restaurant_list_agg.shape[0]+1)

In [928]:
restaurant_list_agg = restaurant_list_agg.reset_index(drop=True)
restaurant_list_agg.head()

,categories,count
0,French Restaurant,175
1,Bakery,171
2,Café,99
3,Coffee Shop,70
4,Fast Food Restaurant,61


<p>We can see that we have a large variety of restaurants, maybe we can aggregate or drop some of the values that we have here. Here are the observations we can make:</p>
<ul>
    <li>We can already drop values that contains the words "Shop", "Supermarket", "Room", "Bar", "Store", "Café", "Bakery" and various of other stop words</li>
    <li>Restaurant does not tell us much because it is not specific to a type of food or country, we can drop this value</li>
</ul>

In [929]:
def getDropValues(d_list):
    drop_list = pd.DataFrame(columns =['categories', 'count'])
    for i in d_list:
        var = restaurant_list_agg[restaurant_list_agg['categories'].str.contains(i)]
        drop_list = drop_list.append(var)
    return drop_list

In [930]:
drop_list = ['Shop', 'Supermarket', 'Room', 'Bar', 'Store', 'Café', 'Coffee', 'Tea', 'Bakery', 'Grocery', 'Hotel', 'Cocktail', 'Pub', 'Club', 'Lounge', 'Butcher', 'Boutique', 'Bookstore', 'Playground', 'Other Nightlife', 'Deli / Bodega']
temp = getDropValues(drop_list)
drop_values = temp['categories']
restaurant_list_fin = restaurant_list_agg[~restaurant_list_agg['categories'].isin(drop_values)]
restaurant_list_fin.head()
#drop_values

,categories,count
0,French Restaurant,175
4,Fast Food Restaurant,61
5,Bistro,56
6,Italian Restaurant,54
7,Pizza Place,42


<p>Setting categories as index:</p>

In [931]:
restaurant_list_fin = restaurant_list_fin.set_index('categories') 

In [932]:
if 'Diner' in restaurant_list_fin.index:
    restaurant_list_fin.rename(index={'Diner':'American Restaurant'},inplace=True)
if 'Restaurant' in restaurant_list_fin.index:
    restaurant_list_fin = restaurant_list_fin.drop('Restaurant')

#restaurant_list_fin

In [933]:
restaurant_list_fin.shape

(90, 1)

<h3>Aggregation analysis</h3>

<p>In our filtered dataframe we have 90 different types of place in Paris. This is quite a big number. Let's see which are the most common type of restaurants:</p>

In [934]:
restaurant_country = restaurant_list_fin.filter(like='Restaurant', axis=0)
restaurant_country.head()

,count
categories,
French Restaurant,175
Fast Food Restaurant,61
Italian Restaurant,54
Japanese Restaurant,36
Chinese Restaurant,27


<p>We can observe that the five most common types of restaurants in Paris are French Restaurants, Fast Foods, Italian Restaurant, Japanese Restaurant and Chinese.</p>
<p>There is a lot of Italian restaurant as well as Chinese and fast food, which is quite common in big cities around the world, but what is interesting here is the huge popularity of Japanese restaurants, which is not really common in other cities. It is really a particularity of Paris where Japanese culture and food is really popular. We could maybe compare this trend to Indian restaurants in London which are also really popular there.
<p>Let's see the least common type of restaurants in the capital to see what we can find out as well !</p>

In [935]:
#restaurant_country.tail()
restaurant_country_tail = restaurant_country.loc[restaurant_country['count'] == 1]
restaurant_country_tail

,count
categories,
Tapas Restaurant,1
Scandinavian Restaurant,1
Soba Restaurant,1
Cantonese Restaurant,1
Caucasian Restaurant,1
Tibetan Restaurant,1
Szechuan Restaurant,1
Cambodian Restaurant,1
Syrian Restaurant,1


<p>Interesting ! We can see that least common restaurants are most of the type specific regional restaurants that are not largely known (Tibetan, Kurdish, Malay, Dim Sum, Corsican, Provencal, Caucasian, Shanxi...) but there are also some surprising observations that we can make:</p>
It seems that eastern european and northern countries food are not well represented and thus, not very popular compared to meditarrean food which is much more present. In general, there seem to be a bigger audience for Meditarrean, Asian and American food than these categories.</li>

<p>Now it should be interesting to see the most common type of restaurant in each neighborhood. Maybe we can learn something valuable as well.</p>

<h3>Neighborhood Analysis</h3>

<p>Let's reuse our code and associate each restaurant to it's neighborhood and group by type of restaurant:</p>

In [936]:
restaurant_list = pd.DataFrame(columns =['categories','nei_number','count'])
drop_list = ['Shop', 'Supermarket', 'Room', 'Bar', 'Store', 'Café', 'Coffee', 'Tea', 'Bakery', 'Grocery', 'Hotel', 'Cocktail', 'Pub', 'Club', 'Lounge', 'Butcher', 'Boutique', 'Bookstore', 'Playground', 'Other Nightlife', 'Deli / Bodega']
temp = getDropValues(drop_list)
drop_values = temp['categories']

In [937]:
radius = 1000

for i in range(len(district_detail.index)):
    neighborhood_lat = district_detail['Latitude'].iloc[i]
    neighborhood_lon = district_detail['Longitude'].iloc[i]
    url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_lon, 
    radius, 
    LIMIT)

    results = requests.get(url).json()
    try:
        venues = results['response']['venues']
    except:
        print('No venues for: ',i)
    nearby_rest = json_normalize(venues) # flatten JSON

    try:
        nearby_rest =nearby_rest[['name', 'categories']]
        nearby_rest['categories'] = nearby_rest.apply(get_category_type, axis=1)
        nearby_rest = nearby_rest.append({'nei_number':i}, ignore_index=True)
        nearby_rest['nei_number'] = i
        nearby_rest = nearby_rest[~nearby_rest['categories'].isin(drop_values)]
        nearby_rest = nearby_rest.groupby(['nei_number','categories'],as_index=False).count()
        nearby_rest = nearby_rest.rename(columns={"name": "count"})
        nearby_rest = nearby_rest[nearby_rest.categories != 'Restaurant']
        nearby_rest = nearby_rest.sort_values(by=['nei_number', 'count'], ascending=False)
        nearby_rest = nearby_rest.iloc[0]
        
        
        restaurant_list = restaurant_list.append(nearby_rest)
        
    except:
        print('No data for point: ',i)

#restaurant_list.head()

In [938]:
#pd.set_option('display.max_rows', restaurant_list.shape[0]+1)
restaurant_list = restaurant_list.drop_duplicates()
restaurant_list = restaurant_list.reset_index(drop=True)
restaurant_list.head()

,categories,nei_number,count
0,French Restaurant,0,3
1,French Restaurant,1,5
2,Burger Joint,2,3
3,French Restaurant,3,5
4,French Restaurant,4,3


<p>Now we can append this new info in our district_detail dataframe and plot this dataframe with each color being the most popular food in each neighborhood. Let's see what we can find !</p>

In [939]:
district_detail['pop_rest'] = restaurant_list['categories']
district_detail['rest_num'] = restaurant_list['count']

In [940]:
district_detail = district_detail.dropna()
#district_detail = district_detail['pop_rest'].astype('category')
district_detail.head()

,Neighborhood Number,Neighborhood Name,District,Perimeter,Surface,Geometry,Latitude,Longitude,pop_rest,rest_num
0,15.0,Arsenal,4.0,2878.559656,4.872649e+05,"{""type"": ""Polygon"", ""coordinates"": [[[2.368512...",48.851585175,2.36476795387,French Restaurant,3
1,18.0,Jardin-des-Plantes,5.0,4052.729521,7.983894e+05,"{""type"": ""Polygon"", ""coordinates"": [[[2.364561...",48.8419401934,2.35689388962,French Restaurant,5
2,39.0,Porte-Saint-Martin,10.0,3245.891413,6.090347e+05,"{""type"": ""Polygon"", ""coordinates"": [[[2.363917...",48.8712446509,2.36150364735,Burger Joint,3
3,43.0,Roquette,11.0,4973.010557,1.172087e+06,"{""type"": ""Polygon"", ""coordinates"": [[[2.379720...",48.8570640408,2.38036406173,French Restaurant,5
4,46.0,Picpus,12.0,18261.910318,7.205014e+06,"{""type"": ""Polygon"", ""coordinates"": [[[2.411249...",48.8303592424,2.42882681508,French Restaurant,3


<p>(Changing type to category to map restaurant type to integer values)</p>

In [941]:
district_detail['pop_rest'] = district_detail['pop_rest'].astype('category')

In [942]:
r_values = district_detail['pop_rest'].cat.codes
district_detail['pop_rest_ind'] = r_values

In [943]:
x = np.arange(district_detail['pop_rest'].nunique())
#ys = [i + x + (i*x)**2 for i in range(district_detail['pop_rest'].nunique())]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [946]:
map_rest_pop = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, neighborhood, district,pop_rest, pop_rest_ind in zip(district_detail['Latitude'], district_detail['Longitude'], district_detail['Neighborhood Name'], district_detail['District'], district_detail['pop_rest'], district_detail['pop_rest_ind']):
    label = '{}, {}, {}'.format(neighborhood, district, pop_rest)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color=rainbow[pop_rest_ind-1],
        fill=True,
        #fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(map_rest_pop)
map_rest_pop

<p>We can make the following observations:</p>
<ul>
    <li>Fairly enough, most of the neighborhoods have French restaurant as their most common type of restaurant. Especially left shore of Paris where almost every neighborhood most common restaurant is <b>French restaurant</b>, which exceptions of <b>Odéon</b>, <b>Salpêtrière</b> and <b>Saint Victor</b> which are student areas and thus where <b>Fast Food</b> and <b>Burger Joins</b> are more popular and <b>Maison-Blanche</b> where Vietnamese is the most popular thanks to Vietnamese communities living in this neighborhoods</li>
    <li>Right shore seems more diverse and gives room to other type of food</li>
    <li><b>Asian restaurants</b> are the most popular around <b>Folie Méricourt</b></li>
    <li><b>Sandwich places</b> seem to be the most popular on the right shore in the historical center of the city. It is common for parisian workers to take a sandwich at lunch when in a hurry. It seems that this type of restaurant is really popular in the first 5 districts but not so much in other parts of the city, where the classical french restaurant is more frequent there</li>
    <li><b>Italian restaurants</b> and <b>Japanese restaurants</b> are really popular but never the more frequent in any district</li>
    
</ul>

<p>Let's drop french restaurants to see if we can come up with other meaningful observations.</p>

In [987]:
restaurant_list_foreign = pd.DataFrame(columns =['categories','nei_number','count'])
#we add french restaurant to our drop list
drop_list = ['Shop', 'Supermarket', 'Room', 'Bar', 'Store', 'Café', 'Coffee', 'Tea', 'Bakery', 'Grocery', 'Hotel', 'Cocktail', 'Pub', 'Club', 'Lounge', 'Butcher', 'Boutique', 'Bookstore', 'Playground', 'Other Nightlife', 'Deli / Bodega', 'French Restaurant']
temp = getDropValues(drop_list)
drop_values = temp['categories']

In [988]:
radius = 1000

for i in range(len(district_detail.index)):
    neighborhood_lat = district_detail['Latitude'].iloc[i]
    neighborhood_lon = district_detail['Longitude'].iloc[i]
    url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_lon, 
    radius, 
    LIMIT)

    results = requests.get(url).json()
    try:
        venues = results['response']['venues']
    except:
        print('No venues for: ',i)
    nearby_rest = json_normalize(venues) # flatten JSON

    try:
        nearby_rest =nearby_rest[['name', 'categories']]
        nearby_rest['categories'] = nearby_rest.apply(get_category_type, axis=1)
        nearby_rest = nearby_rest.append({'nei_number':i}, ignore_index=True)
        nearby_rest['nei_number'] = i
        nearby_rest = nearby_rest[~nearby_rest['categories'].isin(drop_values)]
        nearby_rest = nearby_rest.groupby(['nei_number','categories'],as_index=False).count()
        nearby_rest = nearby_rest.rename(columns={"name": "count"})
        nearby_rest = nearby_rest[nearby_rest.categories != 'Restaurant']
        nearby_rest = nearby_rest.sort_values(by=['nei_number', 'count'], ascending=False)
        nearby_rest = nearby_rest.iloc[0]
        
        
        restaurant_list_foreign = restaurant_list_foreign.append(nearby_rest)
        
    except:
        print('No data for point: ',i)

restaurant_list_foreign.head()

,categories,nei_number,count
14,Sandwich Place,0,3
3,Fast Food Restaurant,1,3
2,Burger Joint,2,3
4,Fast Food Restaurant,3,3
0,Bistro,4,2


In [990]:
restaurant_list_foreign = restaurant_list_foreign.drop_duplicates()
restaurant_list_foreign = restaurant_list_foreign.reset_index(drop=True)
district_detail_foreign = district_detail
district_detail_foreign = district_detail_foreign.drop(columns=['pop_rest', 'rest_num', 'pop_rest_ind'])
district_detail_foreign['pop_rest'] = restaurant_list_foreign['categories']
district_detail_foreign['rest_num'] = restaurant_list_foreign['count']
district_detail_foreign = district_detail_foreign.dropna()
district_detail_foreign['pop_rest'] = district_detail_foreign['pop_rest'].astype('category')
r_values = district_detail_foreign['pop_rest'].cat.codes
district_detail_foreign['pop_rest_ind'] = r_values
x = np.arange(district_detail_foreign['pop_rest'].nunique())
ys = [i + x + (i*x)**2 for i in range(district_detail_foreign['pop_rest'].nunique())]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [991]:
map_rest_pop = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, neighborhood, district,pop_rest, pop_rest_ind in zip(district_detail_foreign['Latitude'], district_detail_foreign['Longitude'], district_detail_foreign['Neighborhood Name'], district_detail_foreign['District'], district_detail_foreign['pop_rest'], district_detail_foreign['pop_rest_ind']):
    label = '{}, {}, {}'.format(neighborhood, district, pop_rest)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color=rainbow[pop_rest_ind-1],
        fill=True,
        #fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(map_rest_pop)
map_rest_pop

<p>Now we clearly have different clusters to analyse on which we can base our choice to see the trends of each food type in different paris neighborhoods:
    <ul>
        <li><b>Sandwich places</b> are really popular on right shore in historical districts (1,2,3,11,17)</li>
        <li><b>Burger Joins</b> are popular among student districts (4,5,6)</li>
        <li><b>American restaurants</b> are popular around the seine (1,5)</li>
        <li><b>Asian</b> and <b>Fast Foods</b> are more popular in peripherical districts (8,10,12,13,17,20)</li>
        <li><b>Italian</b> restaurants are the most popular on left shore in center as well as peripherical areas. This corresponds to the latin district of Paris (7,14, 16, 18, 19)</li>
        <li><b>Vietnamese</b> restaurants are the most popular in the south of the city (13)</li>
        <li><b>Chinese</b> restaurants are more popular in the west of the city (15, 16)</li>
        <li><b>Japanese</b> restaurants are popular in the 15th district</li>
    </ul>
    
           
        
            
        

<h3>Location Analysis</h3>

<p>Now say we don't want to be too original and focus our analysis on the 5 most common type of restaurant in Paris and we want to minimise the risk of our investment by opening one of these type of restaurants in Paris. Where should we open it ?</p>
<p>Let's say we want to maximize our potential customer base and find the location in the city center where there is a minimum of competition for our restaurant. We would like to open one of the five most popular type of food in the city in the area that has the least concentration of such restaurants.</p>

<p>Let's create a heatmap of the concentration of restaurants in Paris and see for each type of restaurant the repartition, to see where we should open our restaurant:</p>

In [947]:
neighbors = district_detail[['Latitude', 'Longitude']].values

In [948]:
map_all= folium.Map(location=[latitude, longitude], zoom_start=12)
all_loc = restaurant_list_det[['latitude','longitude']].values
HeatMap(all_loc, radius = 12).add_to(folium.FeatureGroup(name='Heat Map').add_to(map_all))
folium.LayerControl().add_to(map_all)
map_all

<p>We now have a heatmap of the concentrations of restaurants in Paris. We will first try to find for each restaurants in the city center the best neighborhood that maximise the distance from other similar restaurant using K Nearest Neighbors analysis and use the heatmap to define the streets where to find a location.</p>

<h3>French restaurants</h3>

In [949]:
french_restaurants = restaurant_list_det.loc[restaurant_list_det['categories'] == 'French Restaurant']

In [950]:
map_french= folium.Map(location=[latitude, longitude], zoom_start=12)
HeatMap(all_loc, radius = 12).add_to(folium.FeatureGroup(name='Heat Map').add_to(map_french))
folium.LayerControl().add_to(map_french)

for lat, lng, name in zip(french_restaurants['latitude'], french_restaurants['longitude'], french_restaurants['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_opacity=1,
        parse_html=False).add_to(map_french)
map_french

<p>Here are our restaurants in the city, let's find the best location :</p>

In [951]:
french_samples = french_restaurants[['latitude', 'longitude']].reset_index(drop = True).values
# Create our class with labeled cluster
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(french_samples)
# get the distance/index to the nearest neighbor of neighbords data
distances, indexes = neigh.kneighbors(neighbors, 1, return_distance=True)
french_distance = pd.DataFrame(distances)
french_distance.columns = ['Distance']
french_distance['Neighborhood'] = district_detail['Neighborhood Name']
french_distance = french_distance.sort_values(by=['Distance'], ascending = False)
french_distance.head()

,Distance,Neighborhood
73,0.014391,Gare
58,0.012660,La Chapelle
18,0.009647,Salpêtrière
63,0.009551,Vivienne
78,0.009441,Parc-de-Montsouris


<p>Let's focus on the neighborhood Gare for our location:</p>

In [952]:
#define lat and long of selected neighborhood
lat = district_detail.loc[district_detail['Neighborhood Name'] == french_distance['Neighborhood'].iloc[0]]['Latitude']
lon = district_detail.loc[district_detail['Neighborhood Name'] == french_distance['Neighborhood'].iloc[0]]['Longitude']

In [953]:
map_french= folium.Map(location=[lat, lon], zoom_start=15)
HeatMap(all_loc, radius = 12).add_to(folium.FeatureGroup(name='Heat Map').add_to(map_french))
folium.LayerControl().add_to(map_french)
folium.Circle(location=[lat, lon], radius=600, color='white', fill=True, fill_opacity=0.3).add_to(map_french)

for lat, lng, name in zip(french_restaurants['latitude'], french_restaurants['longitude'], french_restaurants['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_opacity=1,
        parse_html=False).add_to(map_french)
map_french

<p>Looks like a fine location for a french restaurant would be around <b>Rue de Patay</b> or <b>Rue Nationale</b>. We now have a fine perimeter away from other french restaurants that we can prospect on the local adverts.</p>

<h3>Fast Food Restaurants</h3>

In [954]:
ff_restaurants = restaurant_list_det.loc[restaurant_list_det['categories'] == 'Fast Food Restaurant']

In [955]:
map_ff= folium.Map(location=[latitude, longitude], zoom_start=12)
HeatMap(all_loc, radius = 12).add_to(folium.FeatureGroup(name='Heat Map').add_to(map_ff))
folium.LayerControl().add_to(map_ff)

for lat, lng, name in zip(ff_restaurants['latitude'], ff_restaurants['longitude'], ff_restaurants['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='purple',
        fill=True,
        fill_opacity=1,
        parse_html=False).add_to(map_ff)
map_ff

<p>Here are our restaurants in the city, let's find the best location:</p>

In [956]:
ff_samples = ff_restaurants[['latitude', 'longitude']].reset_index(drop = True).values
# Create our class with labeled cluster
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(ff_samples)
# get the distance/index to the nearest neighbor of neighbords data
distances, indexes = neigh.kneighbors(neighbors, 1, return_distance=True)
ff_distance = pd.DataFrame(distances)
ff_distance.columns = ['Distance']
ff_distance['Neighborhood'] = district_detail['Neighborhood Name']
ff_distance = ff_distance.sort_values(by=['Distance'], ascending = False)
ff_distance.head()

,Distance,Neighborhood
4,0.018715,Picpus
44,0.018489,Muette
23,0.016154,Invalides
10,0.013309,Ecole-Militaire
53,0.012807,Saint-Germain-l'Auxerrois


<p>Picpus and Muette are far from the city center, so let's focus on Invalides instead.</p>

In [957]:
#define lat and long of selected neighborhood
lat = district_detail.loc[district_detail['Neighborhood Name'] == ff_distance['Neighborhood'].iloc[2]]['Latitude']
lon = district_detail.loc[district_detail['Neighborhood Name'] == ff_distance['Neighborhood'].iloc[2]]['Longitude']

In [958]:
map_ff= folium.Map(location=[lat, lon], zoom_start=15)
HeatMap(all_loc, radius = 12).add_to(folium.FeatureGroup(name='Heat Map').add_to(map_ff))
folium.LayerControl().add_to(map_ff)
folium.Circle(location=[lat, lon], radius=600, color='white', fill=True, fill_opacity=0.3).add_to(map_ff)

for lat, lng, name in zip(ff_restaurants['latitude'], ff_restaurants['longitude'], ff_restaurants['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_opacity=1,
        parse_html=False).add_to(map_ff)
map_ff

<p>Looks like a fine location for a fast food would be around <b>Rue de Bourgogne</b>, <b>Rue Saint Dominique</b> or <b>Rue de Varenne</b>. We now have a fine perimeter away from other fast foods that we can prospect on the local adverts.</p>

<h3>Italian Restaurants</h3>

In [959]:
it_restaurants = restaurant_list_det.loc[restaurant_list_det['categories'] == 'Italian Restaurant']

In [960]:
map_it= folium.Map(location=[latitude, longitude], zoom_start=12)
HeatMap(all_loc, radius = 12).add_to(folium.FeatureGroup(name='Heat Map').add_to(map_it))
folium.LayerControl().add_to(map_it)

for lat, lng, name in zip(it_restaurants['latitude'], it_restaurants['longitude'], it_restaurants['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_opacity=1,
        parse_html=False).add_to(map_it)
map_it

In [961]:
it_samples = it_restaurants[['latitude', 'longitude']].reset_index(drop = True).values
# Create our class with labeled cluster
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(it_samples)
# get the distance/index to the nearest neighbor of neighbords data
distances, indexes = neigh.kneighbors(neighbors, 1, return_distance=True)
it_distance = pd.DataFrame(distances)
it_distance.columns = ['Distance']
it_distance['Neighborhood'] = district_detail['Neighborhood Name']
it_distance = it_distance.sort_values(by=['Distance'], ascending = False)
it_distance.head()

,Distance,Neighborhood
26,0.019558,Amérique
43,0.016836,Pont-de-Flandre
28,0.014871,Odéon
2,0.014371,Porte-Saint-Martin
70,0.013868,Plaine de Monceaux


<p>Let's focus our analysis on Amérique.</p>

In [965]:
#define lat and long of selected neighborhood
lat = district_detail.loc[district_detail['Neighborhood Name'] == it_distance['Neighborhood'].iloc[0]]['Latitude']
lon = district_detail.loc[district_detail['Neighborhood Name'] == it_distance['Neighborhood'].iloc[0]]['Longitude']

In [966]:
map_it= folium.Map(location=[lat, lon], zoom_start=15)
HeatMap(all_loc, radius = 12).add_to(folium.FeatureGroup(name='Heat Map').add_to(map_it))
folium.LayerControl().add_to(map_it)
folium.Circle(location=[lat, lon], radius=600, color='white', fill=True, fill_opacity=0.3).add_to(map_it)

for lat, lng, name in zip(it_restaurants['latitude'], it_restaurants['longitude'], it_restaurants['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_opacity=1,
        parse_html=False).add_to(map_it)
map_it

<p>Looks like the best location for an Italian restaurant would be around Danube. Probably <b>Rue de Mouzaka</b> or <b>Rue David d'Angers</b> would be a good pick.</p>

<h3>Japanese Restaurants</h3>

In [967]:
jap_restaurants = restaurant_list_det.loc[restaurant_list_det['categories'] == 'Japanese Restaurant']

In [968]:
map_jap= folium.Map(location=[latitude, longitude], zoom_start=12)
HeatMap(all_loc, radius = 12).add_to(folium.FeatureGroup(name='Heat Map').add_to(map_jap))
folium.LayerControl().add_to(map_jap)

for lat, lng, name in zip(jap_restaurants['latitude'], jap_restaurants['longitude'], jap_restaurants['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_opacity=1,
        parse_html=False).add_to(map_jap)
map_jap

In [969]:
jap_samples = jap_restaurants[['latitude', 'longitude']].reset_index(drop = True).values
# Create our class with labeled cluster
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(jap_samples)
# get the distance/index to the nearest neighbor of neighbords data
distances, indexes = neigh.kneighbors(neighbors, 1, return_distance=True)
jap_distance = pd.DataFrame(distances)
jap_distance.columns = ['Distance']
jap_distance['Neighborhood'] = district_detail['Neighborhood Name']
jap_distance = jap_distance.sort_values(by=['Distance'], ascending = False)
jap_distance.head()

,Distance,Neighborhood
0,0.026416,Arsenal
46,0.025469,Epinettes
32,0.024916,Saint-Victor
18,0.024148,Salpêtrière
1,0.022164,Jardin-des-Plantes


<p>Looks like the best location is Arsenal.</p>

In [970]:
#define lat and long of selected neighborhood
lat = district_detail.loc[district_detail['Neighborhood Name'] == jap_distance['Neighborhood'].iloc[0]]['Latitude']
lon = district_detail.loc[district_detail['Neighborhood Name'] == jap_distance['Neighborhood'].iloc[0]]['Longitude']

In [971]:
map_jap= folium.Map(location=[lat, lon], zoom_start=15)
HeatMap(all_loc, radius = 12).add_to(folium.FeatureGroup(name='Heat Map').add_to(map_jap))
folium.LayerControl().add_to(map_jap)
folium.Circle(location=[lat, lon], radius=600, color='white', fill=True, fill_opacity=0.3).add_to(map_jap)

for lat, lng, name in zip(jap_restaurants['latitude'], jap_restaurants['longitude'], jap_restaurants['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_opacity=1,
        parse_html=False).add_to(map_jap)
map_jap

<p>For a japanese restaurant we should pick a street around Sully Morland such as <b>Boulevard Morland</b> or <b>Boulevard Bourdon</b>.</p>

<h3>Chinese Restaurants</h3>

In [972]:
ch_restaurants = restaurant_list_det.loc[restaurant_list_det['categories'] == 'Chinese Restaurant']

In [973]:
map_ch= folium.Map(location=[latitude, longitude], zoom_start=12)
HeatMap(all_loc, radius = 12).add_to(folium.FeatureGroup(name='Heat Map').add_to(map_ch))
folium.LayerControl().add_to(map_ch)

for lat, lng, name in zip(ch_restaurants['latitude'], ch_restaurants['longitude'], ch_restaurants['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='black',
        fill=True,
        fill_opacity=1,
        parse_html=False).add_to(map_ch)
map_ch

In [974]:
ch_samples = ch_restaurants[['latitude', 'longitude']].reset_index(drop = True).values
# Create our class with labeled cluster
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(ch_samples)
# get the distance/index to the nearest neighbor of neighbords data
distances, indexes = neigh.kneighbors(neighbors, 1, return_distance=True)
ch_distance = pd.DataFrame(distances)
ch_distance.columns = ['Distance']
ch_distance['Neighborhood'] = district_detail['Neighborhood Name']
ch_distance = ch_distance.sort_values(by=['Distance'], ascending = False)
ch_distance.head()

,Distance,Neighborhood
5,0.025636,Plaisance
6,0.025452,Quinze-Vingts
42,0.024429,Petit-Montrouge
69,0.023364,Montparnasse
41,0.021430,Notre-Dame-des-Champs


In [975]:
#define lat and long of selected neighborhood
lat = district_detail.loc[district_detail['Neighborhood Name'] == ch_distance['Neighborhood'].iloc[0]]['Latitude']
lon = district_detail.loc[district_detail['Neighborhood Name'] == ch_distance['Neighborhood'].iloc[0]]['Longitude']

In [976]:
map_ch= folium.Map(location=[lat, lon], zoom_start=15)
HeatMap(all_loc, radius = 12).add_to(folium.FeatureGroup(name='Heat Map').add_to(map_ch))
folium.LayerControl().add_to(map_ch)
folium.Circle(location=[lat, lon], radius=600, color='white', fill=True, fill_opacity=0.3).add_to(map_ch)

for lat, lng, name in zip(ch_restaurants['latitude'], ch_restaurants['longitude'], ch_restaurants['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_opacity=1,
        parse_html=False).add_to(map_ch)
map_ch

<p>The best location for a Chinese restaurant would be around <b>Rue d'Alésia</b> or <b>Rue Didot</b>.</p>